In [3]:
from multiprocessing.dummy import Pool, Queue
import lxml
import tldextract
from tqdm import tqdm
from time import sleep
from bs4 import BeautifulSoup
from bs4.element import Comment
from contextlib import ExitStack
from typing import Generator, Dict, Any
import gzip
import pandas as pd
import codecs
import sys
import os
import json
import re
import numpy as np
from os import listdir
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import GermanStemmer, EnglishStemmer, RussianStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise_distances
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from time import time
import re
import warnings
warnings.filterwarnings("ignore")

In [4]:
regex_symbols = re.compile(r'[^a-zA-Z0-9а-яА-Я\u00E4\u00F6\u00FC\u00C4\u00D6\u00DC\u00df]')
stopwords_new = ['http', 'url', 'img','html', 'https', 'org', 'www', 'jpg', 'png', 'net','com','php', 'uid','src', 'ahttp', 'index', 'htm']
pattern = re.compile(r'\b(' + r'|'.join(stopwords_new) + r')\b\s*')
shortword = re.compile(r'\W*\b\w{1,2}\b')
musor = re.compile(r'\d{1}(\w{2})\d{2,4}')

In [5]:
stemmer_ru = RussianStemmer()
stemmer_eng = EnglishStemmer()
stemmer_ger = GermanStemmer()

In [6]:
stop_words = set(stopwords.words(['english', 'russian', 'german']))

In [7]:
my_position_start = 1
my_position_end = 28027
queue = Queue() # очередь ссылок на книги
for i in range(my_position_start, my_position_end):
    queue.put(i)

In [8]:
#split title to words
def split_title(title):
    words = nltk.word_tokenize(title)
    without_extra_words = [stemmer_ru.stem(stemmer_eng.stem(stemmer_ger.stem(word))) for word in words if word not in stop_words]
    return without_extra_words

In [9]:
#split text to words
def split_text(text):
    words = nltk.word_tokenize(text)
    without_extra_words = [stemmer_ru.stem(stemmer_eng.stem(stemmer_ger.stem(word))) for word in words if word not in stop_words]
    return without_extra_words

In [10]:
#returns words which title contains and text contains
def get_content(title, text, n=None):
    title_words = split_title(title)
    text_words = split_text(text)
    if n is not None:
        vectorizer = CountVectorizer().fit(text_words)
        tmp_words_matrix = vectorizer.transform(text_words)
        tmp_words_count = np.sum(tmp_words_matrix, axis=0)
        tmp_words = [(word, tmp_words_count[0, ind]) for word, ind in vectorizer.vocabulary_.items() if len(word) > 2]
        tmp_words = sorted(tmp_words, reverse=True, key=lambda x: x[1])
        tmp_words = [word[0] for word in tmp_words[:n]]
        doc_words = title_words + tmp_words
    else:
        doc_words = title_words + text_words
    return doc_words

In [11]:
def process_doc(number_of_doc):
    with open('./content/{:d}.dat'.format(number_of_doc), encoding='utf-8') as file:
        text_checker = {}
        html_test = file.read()
        soup = BeautifulSoup(html_test, 'html.parser')
        if soup.title:
            title_name = soup.title.text
            title_name = regex_symbols.sub(" ", title_name)
            title_name = re.sub("\s\s+" , " ", title_name)
        else:
            title_name = ' '
        url = soup.text[:soup.text.index('\n')]
        url = tldextract.extract(url)
        url = url.domain + '.' + url.suffix
        text = soup.text[soup.text.index('\n'):].lower()
        text = regex_symbols.sub(" ", text)
        text = shortword.sub(" ", text)
        text = pattern.sub(" ", text)
        text = musor.sub(' ', text)
        text = re.sub("\s\s+" , " ", text)
        text = re.sub('\xa0|\xad', ' ', text)
        content = get_content(title_name, text, n=25)
        text_checker[number_of_doc] = [url] + content
        return text_checker

In [16]:
def process_all_docs(i):
    with gzip.open('data2/part_{:05d}.jsonl.gz'.format(i), mode='wb') as f_json:
        f_json = codecs.getwriter('utf8')(f_json)

        while not queue.empty():
            try:
                id_new = queue.get()
                record = process_doc(id_new)
            except Exception as e:
                print(id_new, file=sys.stderr)
                print(e, file=sys.stderr)
            record_str = json.dumps(record, ensure_ascii=False)
            print(record_str, file=f_json)

            # счетчик должен атомарно обновиться
            with lock:
                pbar.update(1)


with Pool(processes=8) as pool, tqdm(total=queue.qsize()) as pbar:
    lock = pbar.get_lock()
    pool.map(process_all_docs, range(pool._processes))

  2%|█▏                                                                          | 423/28026 [04:17<1:39:20,  4.63it/s]426
[Errno 22] Invalid argument: './content/426.dat'
  5%|███▋                                                                       | 1386/28026 [13:35<4:25:44,  1.67it/s]1393
[Errno 22] Invalid argument: './content/1393.dat'
  7%|█████▌                                                                     | 2067/28026 [19:36<1:03:20,  6.83it/s]2073
empty vocabulary; perhaps the documents only contain stop words
 12%|████████▉                                                                  | 3337/28026 [30:45<6:38:00,  1.03it/s]3345
[Errno 22] Invalid argument: './content/3345.dat'
 14%|██████████                                                                | 3792/28026 [34:07<10:03:55,  1.50s/it]3800
[Errno 22] Invalid argument: './content/3800.dat'
 22%|████████████████▎                                                          | 6078/28026 [52:17<2:31:59,  2.41it/s

In [111]:
def records_reader(dirname: str) -> Generator[Dict[str, Any], None, None]:
    with ExitStack() as stack:
        files = [stack.enter_context(gzip.open(dirname + '/' + i, mode='rb')) for i in tqdm(listdir(dirname))]
        for j in files:
            d = codecs.getreader('utf8')(j)
            for k in d:
                yield json.loads(k)

In [256]:
df = pd.DataFrame(records_reader('data_bad'))
#df.to_csv('prom_res.csv', index=False)

100%|███████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 160.42it/s]


In [163]:
set([i for i in range(1, 28027)]) - set(np.unique(df.columns.values.astype('int')))

{2073, 5120, 10114, 14030, 19562, 24245}

In [242]:
df1 = pd.DataFrame(columns=['id', 'words'])

In [243]:
df1

,id,words


In [257]:
dicter = {}

In [258]:
for i in range(df.shape[0]):
    for j, k in df.loc[i][~df.loc[i].isna()].items():
        dicter[str(j)] = ' '.join(k)

In [259]:
dicter

{'5120': 'nukucihumun.tk сумм подотчетн лиц организац счет работник отчет расход расчет средств денежн наличн выда учет выдач командировк котор документ авансов сотрудник',
 '10114': 'onliner.by memb профил пользовател лет ден офлайн сайт техник рожден город отправ личн сообщен август пленк senior окн средств neman очистк',
 '19562': 'mysonce.ru скача gta samp e торрент gta samp скача игр san andrea торрент multiplay сервер верс как папк файл sa da кача гта grand theft auto',
 '14030': 'onliner.by объявлен memb год пользовател autoc senior масл техник офлайн сайт ден рожден город профил отправ личн сообщен лет цен merc',
 '24245': 'sci-article.ru анализ нарушен иммунологическ реактивн у дет с заболеван почек дет cd лимфоцит кров хроническ групп иммунологическ показател гломерулонефрит воспалительн процесс стат активац ig анализ помощ иммун корреляцион числ сывороточн'}

In [276]:
df1

,id,words
0,1,zrenielib.ru м б аншин центр репродукц генетик...
1,17,nashizubki.ru современ стоматолог кто так стом...
2,18,yaplakal.com есл счетчик яплакал счетчик плат ...
3,23,det-sad45.ru прошивк dexp инструкц прошивк dex...
4,25,tks.ru медицинск издел формулировк нов ру арх ...
...,...,...
28020,5120,nukucihumun.tk сумм подотчетн лиц организац сч...
28021,10114,onliner.by memb профил пользовател лет ден офл...
28022,19562,mysonce.ru скача gta samp e торрент gta samp с...
28023,14030,onliner.by объявлен memb год пользовател autoc...


In [266]:
list(dicter.values())[0]

'nukucihumun.tk сумм подотчетн лиц организац счет работник отчет расход расчет средств денежн наличн выда учет выдач командировк котор документ авансов сотрудник'

In [275]:
df1.loc[28024] = [int(list(dicter.keys())[4])] + [list(dicter.values())[4]]

In [244]:
for j, k in enumerate(dicter.items()):
    df1.loc[j] = [int(k[0])] + [k[1]]

In [296]:
df2 = pd.read_csv('train_groups.csv')

In [283]:
df1 = df1.rename(columns={'id': 'doc_id'})

In [291]:
df3 = pd.merge(df2, df1, how='left', on='doc_id')

In [293]:
df3.to_csv('test_groups.csv')

In [321]:
df2.columns

Index(['Unnamed: 0', 'pair_id', 'group_id', 'doc_id', 'target', 'words'], dtype='object')

In [344]:
df2 = df2.drop('Unnamed: 0', axis=1)

In [346]:
df2.to_csv('test_groups.csv', index=False)

In [347]:
df2 = pd.read_csv('test_groups.csv')

In [313]:
df_train = pd.read_csv('./train_groups.csv')

In [314]:
df_test = pd.read_csv('./test_groups.csv')

In [316]:
featss = [i for i in df_train.columns.values][5:] #in case features are in test/train

In [317]:
df_train.drop(featss,axis=1, inplace=True)

In [318]:
df_test.drop(featss,axis=1, inplace=True)

In [277]:
#df_merged = pd.concat([df_train, df_test])

In [324]:
tf_vect = TfidfVectorizer()

In [325]:
train_mas = tf_vect.fit_transform(df_train.words.values)

In [280]:
#merged_mas = tf_vect.fit_transform(df_merged.words.values)

In [326]:
help_mas = np.empty((df_train.shape[0], 25*5))

In [281]:
#help_mas_merged = np.empty((df_merged.shape[0], 25*5)) # for merged

In [327]:
for i in df_train.groupby('group_id'):
    ind = i[1].index.values
    values = train_mas[ind, :]
    distances = np.partition(pairwise_distances(values, metric='cosine'),25, axis=1)[:,:25]
    meds = np.tile(np.median(distances, axis=0), distances.shape[0]).reshape(distances.shape[0], -1)
    maxs = np.tile(np.max(distances, axis=0), distances.shape[0]).reshape(distances.shape[0], -1)
    disps = np.tile(np.std(distances, axis=0), distances.shape[0]).reshape(distances.shape[0], -1)
    means = np.tile(np.mean(distances, axis=0), distances.shape[0]).reshape(distances.shape[0], -1)
    help_mas[ind, :] = np.concatenate((distances, meds, maxs,disps, means), axis=1)

ValueError: kth(=30) out of bounds (26)

In [282]:
#for merged
'''
for i in df_merged.groupby('group_id'):
    ind = i[1].index.values
    values = merged_mas[ind, :]
    distances = np.partition(pairwise_distances(values, metric='cosine'),25, axis=1)[:,:25]
    meds = np.tile(np.median(distances, axis=0), distances.shape[0]).reshape(distances.shape[0], -1)
    maxs = np.tile(np.max(distances, axis=0), distances.shape[0]).reshape(distances.shape[0], -1)
    disps = np.tile(np.std(distances, axis=0), distances.shape[0]).reshape(distances.shape[0], -1)
    means = np.tile(np.mean(distances, axis=0), distances.shape[0]).reshape(distances.shape[0], -1)
    help_mas_merged[ind, :] = np.concatenate((distances, meds, maxs,disps, means), axis=1)
'''

In [283]:
#df_help_merged = pd.DataFrame(help_mas_merged)

In [285]:
#df_help_merged.columns = ['fit' + str(i) for i in df_help_merged.columns.values]

In [165]:
df_help = pd.DataFrame(help_mas)

In [295]:
#df_train = pd.merge(df_train, df_help_merged.loc[df_train.index.values] ,how='left', left_index=True, right_index=True)

In [297]:
#df_test = pd.merge(df_test, df_help_merged.loc[df_test.index.values] ,how='left', left_index=True, right_index=True)

In [299]:
#df_test.to_csv('test_groups1.csv', index=False)
#df_train.to_csv('train_groups1.csv', index=False)

In [166]:
df_help.columns = ['fit' + str(i) for i in df_help.columns.values]

In [173]:
df_train = pd.merge(df_train, df_help ,how='left', left_index=True, right_index=True)

In [175]:
df_train.to_csv('train_groups.csv', index=False)

In [183]:
df_test = pd.read_csv('./test_groups.csv')

In [185]:
test_mas = tf_vect.transform(df_test.words.values)
help_mas1 = np.empty((df_test.shape[0], 25*5))
for i in df_test.groupby('group_id'):
    ind = i[1].index.values
    values = test_mas[ind, :]
    distances = np.partition(pairwise_distances(values, metric='cosine'),25, axis=1)[:,:25]
    meds = np.tile(np.median(distances, axis=0), distances.shape[0]).reshape(distances.shape[0], -1)
    maxs = np.tile(np.max(distances, axis=0), distances.shape[0]).reshape(distances.shape[0], -1)
    disps = np.tile(np.std(distances, axis=0), distances.shape[0]).reshape(distances.shape[0], -1)
    means = np.tile(np.mean(distances, axis=0), distances.shape[0]).reshape(distances.shape[0], -1)
    help_mas1[ind, :] = np.concatenate((distances, meds, maxs,disps, means), axis=1)
df_help = pd.DataFrame(help_mas1)
df_help.columns = ['fit' + str(i) for i in df_help.columns.values]
df_test = pd.merge(df_test, df_help ,how='left', left_index=True, right_index=True)

In [187]:
df_test.to_csv('test_groups.csv', index=False)

In [300]:
df_train = pd.read_csv('train_groups1.csv')
traingroups_titledata = {}
my_feat = [i for i in df_train.columns.values][5:]
for i in range(len(df_train)):
    new_doc = df_train.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    target = new_doc['target']
    features = new_doc[my_feat]
    title = new_doc['words']
    if doc_group not in traingroups_titledata:
        traingroups_titledata[doc_group] = []
    traingroups_titledata[doc_group].append((doc_id, title, features, target))

In [301]:
y_train = []
X_train = []
groups_train = []
for new_group in traingroups_titledata:
    docs = traingroups_titledata[new_group]
    for k, (doc_id, title, features, target_id) in enumerate(docs):
        y_train.append(target_id)
        groups_train.append(new_group)
        all_dist = []
        words = set(title.strip().split())
        for j in range(0, len(docs)):
            if k == j:
                continue
            doc_id_j, title_j, feat_j, target_j = docs[j]
            words_j = set(title_j.strip().split())
            all_dist.append(len(words.intersection(words_j)))
        X_train.append(sorted(all_dist, reverse=True)[0:25] + list(features)    )
X_train = np.array(X_train)
y_train = np.array(y_train)
groups_train = np.array(groups_train)
print (X_train.shape, y_train.shape, groups_train.shape)

(11689, 150) (11689,) (11689,)


In [302]:
df_test = pd.read_csv('test_groups1.csv')
testgroups_titledata = {}
for i in range(len(df_test)):
    new_doc = df_test.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    features = new_doc[my_feat]
    title = new_doc['words']
    if doc_group not in testgroups_titledata:
        testgroups_titledata[doc_group] = []
    testgroups_titledata[doc_group].append((doc_id, title, features))

In [303]:
X_test = []
groups_test = []
for new_group in testgroups_titledata:
    docs = testgroups_titledata[new_group]
    for k, (doc_id, title, features) in enumerate(docs):
        groups_test.append(new_group)
        all_dist = []
        words = set(title.strip().split())
        for j in range(0, len(docs)):
            if k == j:
                continue
            doc_id_j, title_j, feat_j = docs[j]
            words_j = set(title_j.strip().split())
            all_dist.append(len(words.intersection(words_j)))
        X_test.append(sorted(all_dist, reverse=True)[0:25] + list(features)   )
X_test = np.array(X_test)
groups_test = np.array(groups_test)
print (X_test.shape, groups_test.shape)

(16627, 150) (16627,)


In [322]:
warnings.filterwarnings("ignore")

In [304]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
kf = StratifiedKFold(n_splits=5, shuffle=True)

In [305]:
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.1)

In [306]:
kf = StratifiedKFold(n_splits=5, shuffle=True)

def validate_model(estimator, params, validate_param, vals):
    score = 0
    for t in tqdm(vals):
        params[validate_param] = t
        tempScore = 0
        for train_split, test_split in kf.split(X_train, y_train):
            X_tr = X_train[train_split]
            y_tr = y_train[train_split]
            X_tst = X_train[test_split]
            y_tst = y_train[test_split]
            clf = estimator(**params)
            clf.fit(X_tr, y_tr)
            my_clf_pred = clf.predict(X_tst)
            tempScore += f1_score(y_tst, my_clf_pred)
        if (tempScore > score):
            score = tempScore
            result = t
    print(validate_param, ' = ', result, ' score= ', score / 5)
    return result

In [307]:
params = {'objective': 'binary:logistic'}
params['base_score'] = validate_model(xgb.XGBClassifier, params, 'base_score', np.linspace(0.0001,0.99999,10))
params['n_estimators'] = validate_model(xgb.XGBClassifier, params, 'n_estimators', 
                                            range(20,100,5))
params['colsample_bytree'] = validate_model(xgb.XGBClassifier, params, 'colsample_bytree', 
                                            np.linspace(0.0001,0.999,10))
params['scale_pos_weight'] = validate_model(xgb.XGBClassifier, params, 'scale_pos_weight', 
                                            np.linspace(2.0,3.0,10))
params['alpha'] = validate_model(xgb.XGBClassifier, params, 'alpha', 
                                            np.linspace(0.0001,1.0,10))
params['min_child_weight'] = validate_model(xgb.XGBClassifier, params, 'min_child_weight', 
                                            np.linspace(0.0001,1.0,10))
paramsLogistic = params

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

[17:29:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:29:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:29:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:29:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

 10%|████████▎                                                                          | 1/10 [00:01<00:13,  1.48s/it]

[17:29:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:29:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:29:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:29:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

 20%|████████████████▌                                                                  | 2/10 [00:11<00:33,  4.15s/it]

...

[17:35:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:35:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:35:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:35:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [00:39<00:04,  4.44s/it]

[17:35:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:35:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:35:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:35:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:43<00:00,  4.39s/it]

min_child_weight  =  0.0001  score=  0.7634628955070301


In [248]:
params = {'booster': 'gbtree'}
params['n_estimators'] = validate_model(xgb.XGBClassifier, params, 'n_estimators', 
                                            range(20,100,5))
params['colsample_bytree'] = validate_model(xgb.XGBClassifier, params, 'colsample_bytree', 
                                            np.linspace(0.0001,0.999,10))
params['scale_pos_weight'] = validate_model(xgb.XGBClassifier, params, 'scale_pos_weight', 
                                            np.linspace(2.0,3.0,10))
params['alpha'] = validate_model(xgb.XGBClassifier, params, 'alpha', 
                                            np.linspace(0.0001,1.0,10))
params['min_child_weight'] = validate_model(xgb.XGBClassifier, params, 'min_child_weight', 
                                            np.linspace(0.0001,1.0,10))
paramsGbTree = params

  0%|                                                                                           | 0/16 [00:00<?, ?it/s]C:\Users\artemii\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:32:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:32:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:32:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:32:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

  6%|█████▏                                                                             | 1/16 [00:02<00:30,  2.01s/it]C:\Users\artemii\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:32:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:32:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:32:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:32:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

 12%|██████████▍                                                                        | 2/16 [00:04<00:30,  2.15s/it]C:\Users\artemii\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[15:38:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:38:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:38:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:38:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [01:05<00:07,  7.24s/it]C:\Users\artemii\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:38:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:38:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:38:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:38:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:12<00:00,  7.28s/it]

min_child_weight  =  0.5556  score=  0.786903529708463


In [310]:
clf = xgb.XGBClassifier(**paramsLogistic)
clf.fit(X_train, y_train)

[17:36:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(alpha=0.2223, base_score=0.4444955555555556, booster='gbtree',
              colsample_bylevel=1, colsample_bynode=1,
              colsample_bytree=0.22207777777777776, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=0.0001, missing=nan, monotone_constraints='()',
              n_estimators=95, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0.222299993, reg_lambda=1,
              scale_pos_weight=2.6666666666666665, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [311]:
f1_score(clf.predict(X_test), y_test)

0.7727272727272728

In [236]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [237]:
clf = xgb.XGBClassifier(**paramsLogistic)
clf.fit(X_train, y_train)

C:\Users\artemii\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:18:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(alpha=0.5556, base_score=0.4444955555555556, booster='gbtree',
              colsample_bylevel=1, colsample_bynode=1,
              colsample_bytree=0.11108888888888889, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=0.33340000000000003, missing=nan,
              monotone_constraints='()', n_estimators=70, n_jobs=8,
              num_parallel_tree=1, random_state=0, reg_alpha=0.555599988,
              reg_lambda=1, scale_pos_weight=2.3333333333333335, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [238]:
predict = clf.predict(X_test)

In [240]:
df_test['target'] = predict

In [312]:
df_train

,pair_id,group_id,doc_id,target,words,fit0,fit1,fit2,fit3,fit4,...,fit115,fit116,fit117,fit118,fit119,fit120,fit121,fit122,fit123,fit124
0,1,1,15731,0,automn.ru ваз зам подшипник ступиц нив подшипн...,0.513324,0.537759,0.000000,0.500693,0.577466,...,0.813664,0.80683,0.818707,0.792901,0.816379,0.833170,0.849067,0.850816,0.858552,0.865540
1,2,1,14829,0,tiu.ru ваз опт соч сравн цен куп потребительск...,0.730339,0.494394,0.780583,0.000000,0.705516,...,0.813664,0.80683,0.818707,0.792901,0.816379,0.833170,0.849067,0.850816,0.858552,0.865540
2,3,1,15764,0,drom.ru куп ступиц лад калин трансмисс переход...,0.694028,0.654646,0.000000,0.656656,0.738771,...,0.813664,0.80683,0.818707,0.792901,0.816379,0.833170,0.849067,0.850816,0.858552,0.865540
3,4,1,17669,0,carobka.su классик ваз тольятт дааз вис задн с...,0.862569,0.858598,0.866645,0.000000,0.853833,...,0.813664,0.80683,0.818707,0.792901,0.816379,0.833170,0.849067,0.850816,0.858552,0.865540
4,5,1,14852,0,cartore.ru ступиц нив — зам подшипник сво рук ...,0.446986,0.577466,0.508407,0.611436,0.000000,...,0.813664,0.80683,0.818707,0.792901,0.816379,0.833170,0.849067,0.850816,0.858552,0.865540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11684,11686,129,26672,0,eva.ru ❤★✿★апрелят ❤★✿ врем дат сообщен ответ ...,0.976094,0.964805,0.976156,0.966278,0.947772,...,0.941930,0.94011,0.920953,0.938129,0.934289,0.955894,0.949497,0.959865,0.971866,0.969867
11685,11687,129,25838,0,psychotherapie-ros.at gastebuch http cu url bi...,0.879701,0.932989,0.956457,0.966050,0.953758,...,0.941930,0.94011,0.920953,0.938129,0.934289,0.955894,0.949497,0.959865,0.971866,0.969867
11686,11688,129,25703,0,tumblr.com jizolofej archiv эт котор класс сам...,0.000000,0.904293,0.944259,0.875528,0.916740,...,0.941930,0.94011,0.920953,0.938129,0.934289,0.955894,0.949497,0.959865,0.971866,0.969867
11687,11689,129,27885,0,prodeundi.ru как зовут парн диа шурыгин пуст г...,0.947830,0.985395,0.979757,0.971201,0.976984,...,0.941930,0.94011,0.920953,0.938129,0.934289,0.955894,0.949497,0.959865,0.971866,0.969867


In [241]:
df_test

,pair_id,group_id,doc_id,words,fit0,fit1,fit2,fit3,fit4,fit5,...,fit116,fit117,fit118,fit119,fit120,fit121,fit122,fit123,fit124,target
0,11691,130,6710,youtube.com как прописа админк в кс себ ил дру...,0.472278,0.000000,0.439519,0.429909,0.501693,0.481584,...,0.778401,0.811453,0.812821,0.825117,0.821944,0.836151,0.843767,0.828287,0.852029,1
1,11692,130,4030,v-sampe.ru скача sgl rp доработк слив мод mysq...,0.536240,0.000000,0.398399,0.615161,0.852823,0.854280,...,0.778401,0.811453,0.812821,0.825117,0.821944,0.836151,0.843767,0.828287,0.852029,0
2,11693,130,5561,dream-x.ru как прописа админк кс count strik к...,0.575746,0.561512,0.000000,0.643492,0.474139,0.519859,...,0.778401,0.811453,0.812821,0.825117,0.821944,0.836151,0.843767,0.828287,0.852029,1
3,11694,130,4055,net.ru как прописа прост админк кс админк кс п...,0.429909,0.614445,0.564138,0.000000,0.397117,0.560535,...,0.778401,0.811453,0.812821,0.825117,0.821944,0.836151,0.843767,0.828287,0.852029,1
4,11695,130,4247,o3one.ru подбор админ сервер код арх форум ozo...,0.885942,0.883692,0.891699,0.883172,0.000000,0.820190,...,0.778401,0.811453,0.812821,0.825117,0.821944,0.836151,0.843767,0.828287,0.852029,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16622,28313,309,16637,mail.ru ответ mail ru полезн куша творог утр х...,0.730864,0.678725,0.000000,0.567072,0.595642,0.633593,...,0.550814,0.557325,0.571887,0.553988,0.593749,0.597871,0.595379,0.606126,0.613904,1
16623,28314,309,16759,inmoment.ru творог полезн свойств лечен творог...,0.622992,0.627836,0.574985,0.637606,0.557075,0.621798,...,0.550814,0.557325,0.571887,0.553988,0.593749,0.597871,0.595379,0.606126,0.613904,1
16624,28315,309,15358,edaplus.info творог полезн опасн свойств творо...,0.476313,0.458961,0.492689,0.446254,0.456509,0.531708,...,0.550814,0.557325,0.571887,0.553988,0.593749,0.597871,0.595379,0.606126,0.613904,1
16625,28316,309,17287,mail.ru ответ mail ru чем полез творог творог ...,0.000000,0.335397,0.201235,0.312665,0.340588,0.130777,...,0.550814,0.557325,0.571887,0.553988,0.593749,0.597871,0.595379,0.606126,0.613904,1


In [244]:
df_test[['pair_id', 'target']].to_csv('new_prediction.csv', index=False)